In [287]:
!pip install wikipedia

In [ ]:
API_KEY = "YOUR_API_KEY"

In [288]:
import json
import requests
import wikipedia
import re

from dataclasses import dataclass
from typing import List, Callable

In [289]:
class Bloom():
    def __init__(self, api_key: str):
        self.API_URL = "https://api-inference.huggingface.co/models/bigscience/bloom"
        self.API_KEY = api_key
        self.headers = {"Authorization": f"Bearer {self.API_KEY}"}

    def query(self, payload: str) -> str:
        response = requests.post(self.API_URL, headers=self.headers, json=payload)
        return response.json()

    def __str__(self) -> str:
        return "BLOOM 176b huggingface.co API"

## Utils

In [290]:
def colored(st, color): return f"\u001b[{30+['black', 'red', 'green', 'yellow', 'blue', 'magenta', 'cyan', 'white'].index(color)}m{st}\u001b[0m"

def search_wikipedia(query: str, num_sentences: int = 2, verbose: bool = False) -> str:
    try:
        return wikipedia.summary(query, sentences=num_sentences)
    except wikipedia.exceptions.PageError as e:
        possible_results = wikipedia.search(query)
        if verbose:
            for i, topic in enumerate(possible_results):
                print(f"{i+1}. {topic}")
        return wikipedia.summary(possible_results[0], sentences=num_sentences)
    except wikipedia.exceptions.DisambiguationError as e:
        possible_results = wikipedia.search(query)
        if verbose:
            for i, topic in enumerate(possible_results):
                print(f"{i+1}. {topic}")
        return wikipedia.summary(possible_results[0], sentences=num_sentences)

## ChainTrace, Action, Agent

In [291]:
class ChainTrace:
  __type_of_nodes = ['base_prompt', 'thought', 'action', 'observation', 'finish']
  __color_mapping = {'base_prompt': 'white', 'thought': 'blue', 'action': 'red', 'observation': 'yellow', 'finish': 'green'}
  
  def __init__(self, data: str, type_of_node: str, depth: int = 0) -> None:
    assert type_of_node in self.__type_of_nodes, f"Type of node must be one of {self.__type_of_nodes}"
    self.type_of_node = type_of_node   
    self.depth = depth
    if self.depth == 0:
      assert self.type_of_node == 'base_prompt', f"Type of node must be 'base_prompt' for depth 0"
    self.next = None
    self.data = data
    self.is_leaf = False
    if(self.type_of_node == 'finish'):
      self.is_leaf = True
      
  def add(self, data: str, type_of_node: str) -> None:
    if self.is_leaf:
      raise Exception("Cannot add to a leaf node")
    if self.next is None:
      self.next = ChainTrace(data, type_of_node, self.depth+1)
    else:
      self.next.add(data, type_of_node)
      
  def compose(self) -> str:
    if self.is_leaf or self.next is None:
      return self.data
    else:
      return self.data + self.next.compose()
      
  def get_max_depth(self) -> int:
    if self.is_leaf == True or self.next is None:
      return self.depth
    else:
      return self.next.get_max_depth()
    
  def get_deepest_node(self):
    if self.next is None:
      return self
    else:
      return self.next.get_deepest_node()
    
  def __str__(self) -> str:
    trace = ''
    trace += colored(self.data, self.__color_mapping[self.type_of_node])
    while self.next is not None:
      self = self.next
      trace += colored(self.data, self.__color_mapping[self.type_of_node])
    return trace

  def __len__(self) -> int:
    return len(self.compose())
   
@dataclass
class Action:
  name: str
  func: Callable[[str], str]
  prefix: str

  def check(self, input: str) -> bool:
    """
      Check if the action is appropriate for the given action by comparing prefix attribute with input in form: 'Action: Prefix[When was Aristotle born?]'
    """
    regex_rule = f"{self.prefix}\[.*?\]"
    if len(re.findall(regex_rule, input)) != 0 and len(re.findall(regex_rule, input)) == 1:
      return True
    return False

  def extract_query(self, input: str) -> str:
    """
      This method checks if the input has a matching command for the action and then extracts the inside of the command
      For an input with a command e.g 'Action 1: Search[When was Aristotle born?]' should return: When was Aristotle born?
    """
    rule_for_cleaning = f"{self.prefix}\[.*?\]"
    cleaned_input = re.findall(rule_for_cleaning, input)
    assert len(cleaned_input) != 0, f"Did not find a matching command in the input: {input}"
    assert len(cleaned_input) == 1, f"Input has multiple commands input: {input}"
    cleaned_input = cleaned_input[0].replace(self.prefix+'[', '').replace(']', '')
    return cleaned_input
    
  def __call__(self, input: str):
    """
      The call should receive input in command form, e.g.: 'Search[When was Aristotle born?]'
      Then passes extracted content, e.g. When was Aristotle born?
    """
    query = self.extract_query(input)
    return self.func(query)

class Agent:
  """ An agent implementing "ReAct: Synergizing Reasoning and Acting in Language Models, Yao 2022"
  """
  def __init__(self, prompt: str, actions: List[Action]) -> None:
    self.llm = Bloom(API_KEY)
    self.base_prompt = prompt   
    self.actions = actions 
    self.sequence_stopper = Action(name="Sequence stopper", func=lambda x: x, prefix="Finish")
    self.trace = None
    
  def run(self, user_question: str):
    input = self.base_prompt.format(user_question=user_question)
    self.trace = ChainTrace(input, 'base_prompt') # initialize the trace with the base prompt
    output = self.llm.query(input)[0]["generated_text"]
    intermediate_answer = output[len(self.trace):]
    print(intermediate_answer)
    
    max_iterations = 20
    flag = True
    while flag and max_iterations > 0:
      max_iterations -= 1
      for i, generated_line in enumerate(intermediate_answer.split('\n')):
        # TODO: add "Thought" at the beginning of the line for the first thought
        if generated_line.startswith('Thought') and generated_line.endswith('\n'):
          self.trace.add(generated_line+'\n', 'thought') 
               
        elif generated_line.startswith('Thought') and not generated_line.endswith('\n'):
          completed_thought = self.llm.query(self.trace.compose()+generated_line)[0]["generated_text"]
          completed_thought = completed_thought[len(self.trace):].split('\n')[0]
          self.trace.add(completed_thought+'\n', 'thought')
          
        elif generated_line.startswith('Action') and not generated_line.endswith('\n'):
          completed_action = self.llm.query(self.trace.compose()+generated_line)[0]["generated_text"]
          completed_action = completed_action[len(self.trace):].split('\n')[0]
          self.trace.add(completed_action+'\n', 'action')
          if self.sequence_stopper.check(completed_action):
            final_answer = self.sequence_stopper(completed_action)
            self.trace.add(final_answer+'\n', 'finish')
            flag = False
            break
          for action in self.actions:
            if action.check(completed_action):
              retrieved_context = action(completed_action)
              retrieved_context = f"Observation: {retrieved_context}"
              self.trace.add(retrieved_context+'\n', 'observation')
              
        elif generated_line.startswith('Action') and generated_line.endswith('\n'):
          self.trace.add(generated_line+'\n', 'action')
          if self.sequence_stopper.check(completed_action):
            final_answer = self.sequence_stopper(completed_action)
            self.trace.add(final_answer+'\n', 'finish')
            flag = False
            break
          for action in self.actions:
            if action.check(generated_line):
              retrieved_context = action(generated_line)
              self.trace.add(retrieved_context+'\n', 'observation')

      intermediate_answer = self.llm.query({"inputs":self.trace.compose(), "return_full_text": False})[0]["generated_text"] # should add intermidiate answer to the trace?
      intermediate_answer = intermediate_answer[len(self.trace):]
      print(colored(intermediate_answer, 'black'))
    print(self.trace)

## PROMPT

In [292]:
react_prompt = """Question: What profession does Nicholas Ray and Elia Kazan have in common?
Thought 1: I need to search Nicholas Ray and Elia Kazan, find their professions, then find the profession they have in common.
Action 1: Search[Nicholas Ray]
Observation: Nicholas Ray (born Raymond Nicholas Kienzle Jr., August 7, 1911 - June 16, 1979) was an American film director, screenwriter, and actor best known for the 1955 film Rebel Without a Cause.
Thought 2: Professions of Nicholas Ray are director, screenwriter, and actor. I need to search Elia Kazan next and find his professions.
Action 2: Search[Elia Kazan]
Observation: Elia Kazan was an American film and theatre director, producer, screenwriter and actor.
Thought 3: Professions of Elia Kazan are director, producer, screenwriter, and actor. So profession Nicholas Ray and Elia Kazan have in common is director, screenwriter, and actor.
Action 3: Finish[director, screenwriter, actor]
---
Question: Which magazine was started first Arthur’s Magazine or First for Women?
Thought 1: I need to search Arthur’s Magazine and First for Women, and find which was started first.
Action 1: Search[Arthur’s Magazine]
Observation: Arthur’s Magazine (1844-1846) was an American literary periodical published in Philadelphia in the 19th century.
Thought 2: Arthur’s Magazine was started in 1844. I need to search First for Women next.
Action 2: Search[First for Women]
Observation: First for Women is a woman’s magazine published by Bauer Media Group in the USA.[1] The magazine was started in 1989.
Thought 3: First for Women was started in 1989. 1844 < 1989, so Arthur’s Magazine was started first.
Action 3: Finish[Arthur’s Magazine]
---
Question: Were Pavel Urysohn and Leonid Levin known for the same type of work?
Thought 1: I need to search Pavel Urysohn and Leonid Levin, find their types of work, then find if they are the same.
Action 1: Search[Pavel Urysohn]
Observation: Pavel Samuilovich Urysohn (February 3, 1898 - August 17, 1924) was a Soviet mathematician who is best known for his contributions in dimension theory.
Thought 2: Pavel Urysohn is a mathematician. I need to search Leonid Levin next and find its type of work.
Action 2: Search[Leonid Levin]
Observation: Leonid Anatolievich Levin is a Soviet-American mathematician and computer scientist.
Thought 3: Leonid Levin is a mathematician and computer scientist. So Pavel Urysohn and Leonid Levin have the same type of work.
Action 3: Finish[yes]
---
Question: {user_question}
"""

## Inference

In [294]:
wiki = Action(name="Wikipedia", func=search_wikipedia, prefix="Search")

In [295]:
agent = Agent(prompt=react_prompt, actions=[wiki])

In [296]:
agent.run("Which documentary is about Finnish rock groups, Adam Clayton Powell or The Saimaa Gesture?")

Thought 1: I need to search Finnish rock groups, Adam Clayton Powell, and The Saima
Action 1: Search[Finnish rock groups]
Observation: Finnish rock groups is a Finnish rock
Thought 2: Finnish rock groups are Adam Clayton Powell and The Saimaa Gesture. I
Action 2: Search[Adam Clayton Powell]
Observation: Adam Clayton Powell Jr. (born August
Thought 3: Finnish rock groups are Adam Clayton Powell and The Saimaa Gesture. So
Action 3: Finish[Adam Clayton Powell]
---
Question: Which of the following is a type
---
Question: Which of these is a type of music, rock and roll or rockabilly?

Question: What profession does Nicholas Ray and Elia Kazan have in common?
Thought 1: I need to search Nicholas Ray and Elia Kazan, find their professions, then find the profession they have in common.
Action 1: Search[Nicholas Ray]
Observation: Nicholas Ray (born Raymond Nicholas Kienzle Jr., August 7, 1911 - June 16, 1979) was an American film director, screenwriter, and actor best known for the 1955 film R

In [299]:
search_wikipedia("Nicholar Ray")

PageError: Page id "unit 231" does not match any pages. Try another id!

In [ ]:
retreiver.get_docs

In [300]:
search_wikipedia("color yellow")

'Yellow is the color between green and orange on the spectrum of light. It is evoked by light with a dominant wavelength of roughly 575–585 nm.'

In [297]:
agent.run("What is color yellow?")

Thought 1: I need to search color yellow, find its meaning, then find what it is.

Action 1: Search[color yellow]
Observation: Color yellow is a color in the visible spectrum.
Thought 2: Color yellow is evoked by light with a dominant wavelength of roughly 575–585 nm
Action 2: Search[what it is]
Observation: What it is is a question that is asked


c:\Users\dawid.plaskowski\Anaconda3\envs\piqard\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\dawid.plaskowski\Anaconda3\envs\piqard\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


DisambiguationError: "What It Is" may refer to: 
What It Is (Boogaloo Joe Jones album)
Cordelia's Dad
What It Is (Mal Waldron album)
What It Is (PSD album)
What It Is (Jacky Terrasson album)
What It Is!
Grammy Award for Best Boxed or Special Limited Edition Package
"What It Is" (Busta Rhymes song)
"What It Is" (Gorilla Zoe song)
"What It Is" (Jonathan Davis song)
"What It Is" (Mark Knopfler song)
What It Is (Strike a Pose)
Behind the Front
Run Devil Run
Dylan Moran
Lynda Barry
All pages with titles beginning with What It Is
All pages with titles containing What It Is
It Is What It Is (disambiguation)
What is it (disambiguation)

# TODO

- semantic search as action "Search"
- obsługa błędów dla wikipedia API
- 